<a href="https://colab.research.google.com/github/yhc-666/hate-speech-detection/blob/master/lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import make_scorer
import lightgbm as lgb


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul 16 14:56:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
df_dataset = pd.read_csv('/content/drive/MyDrive/50.007kaggle/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/50.007kaggle/test.csv')
df_dataset_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/train_tfidf_features.csv")
df_test_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/test_tfidf_features.csv")
samplesubmit = pd.read_csv("/content/drive/MyDrive/50.007kaggle/sample_submission.csv")

In [5]:
print(samplesubmit.shape)

(4296, 2)


In [6]:
labels = df_dataset_features.iloc[:,[1]]
features = df_dataset_features.iloc[:,2:]
#y_test = df_test.iloc[:,[1]]
#X_test = df_test_features.iloc[:,1:]

#分训练集测试集
X_train, X_validation, y_train, y_validation = train_test_split(features, labels, test_size=0.05, random_state=1)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

In [7]:
print(y_validation)
print(y_train)
print(X_train)
print(X_validation)

       label
10817      0
6126       1
15049      1
9695       0
9508       0
...      ...
14580      0
3085       1
13330      1
15540      1
3933       1

[860 rows x 1 columns]
       label
6821       1
9576       0
4952       1
13085      1
14621      0
...      ...
7813       0
10955      0
5192       0
12172      0
235        1

[16324 rows x 1 columns]
         0    1    2    3    4    5    6    7    8    9  ...  4990  4991  \
6821   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
9576   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4952   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
13085  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
14621  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
7813   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
10955  0.0  0.0  0.0  0.0  0.0

In [8]:
def my_custom_loss_func(y_true, y_pred):
    pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
    neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
    macro_f1 = (pos_f1 + neg_f1)/2  
    return macro_f1

macro_f1 = make_scorer(my_custom_loss_func, greater_is_better=False)

In [11]:
params = {
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'binary',  # 目标函数
    'max_depth': -1,
    'metric': 'macro_f1',  # 评估函数
    'num_leaves': 300,  # 叶子节点数
    'learning_rate': 0.01,  # 学习速率
    'feature_fraction': 0.9,  # 建树的特征选择比例
    'bagging_fraction': 0.8,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'lambda_l2': 0.01,
    'nthread' : -1,
    'verbose': 1  
}

In [12]:
gbm = lgb.train(params, lgb_train, valid_sets=lgb_eval)

KeyboardInterrupt: ignored

In [ ]:
y_validation, y_pred_validation = y_validation, gbm.predict(X_validation)
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  
print("validation_acc: %.4g"% metrics.accuracy_score(y_validation, y_pred_validation))

0.6399999999999999
0.729495669893021
validation_macro_f1 : 0.6847
validation_acc: 0.6911


In [ ]:
X_test = df_test_features.iloc[:,1:]

In [ ]:
print(X_test)

        0    1    2    3    4    5    6    7    8    9  ...  4990  4991  4992  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
4291  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4292  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4293  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4294  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4295  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

      4993  4994  4995  499

In [ ]:
y_test = gbm.predict(X_test)

In [ ]:
print(y_test)

[1 1 1 ... 1 0 0]


In [ ]:
x = df_test
x = x.set_index("id")
x = x.drop("post", axis = 1)
x["label"] = y_test
print(x)
x.to_csv("/content/drive/MyDrive/50.007kaggle/submit.csv")

       label
id          
17185      1
17186      1
17187      1
17188      0
17189      0
...      ...
21476      1
21477      1
21478      1
21479      0
21480      0

[4296 rows x 1 columns]
